In [ ]:
!pip install diffusers
!pip install gradio
import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import gradio as gr
import os

# Set up device and data type
device = "cuda"
dtype = torch.float16

# Initialize MotionAdapter and AnimateDiffPipeline
step = 4  # Options: [1,2,4,8]
repo = "ByteDance/AnimateDiff-Lightning"
ckpt = f"animatediff_lightning_{step}step_diffusers.safetensors"
base = "emilianJR/epiCRealism"  # Choose your favorite base model.

adapter = MotionAdapter().to(device, dtype)
adapter.load_state_dict(load_file(hf_hub_download(repo, ckpt), device=device))
pipe = AnimateDiffPipeline.from_pretrained(base, motion_adapter=adapter, torch_dtype=dtype).to(device)
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config, timestep_spacing="trailing", beta_schedule="linear")

# Define the Gradio interface function
def generate_animation(prompt, guidance_scale, width, height):
    try:
        output = pipe(prompt=prompt, guidance_scale=guidance_scale, num_inference_steps=step)

        # Debug: Check if frames are generated
        if not output or not hasattr(output, 'frames') or not output.frames:
            return "Error: No frames were generated."

        # Resize frames to the specified width and height
        resized_frames = [frame.resize((width, height)) for frame in output.frames[0]]

        # Export to GIF
        gif_path = "/content/animation.gif"
        export_to_gif(resized_frames, gif_path)

        # Debug: Check if GIF was created
        if not os.path.exists(gif_path):
            return "Error: GIF was not created."

        return gif_path

    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio interface
interface = gr.Interface(
    fn=generate_animation,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="Describe the animation you want to generate"),
        gr.Slider(label="Guidance Scale", minimum=0.5, maximum=10.0, step=0.5, value=1.0),
        gr.Slider(label="Width (px)", minimum=128, maximum=1024, step=64, value=512),
        gr.Slider(label="Height (px)", minimum=128, maximum=1024, step=64, value=512)
    ],
    outputs=gr.Image(type="filepath", label="Generated Animation"),
    title="AnimateDiff: Generate Animations",
    description="Enter a text prompt to generate an animation using the AnimateDiff pipeline."
)

# Launch the Gradio app in a non-blocking mode
interface.launch(share=True, debug=True)



Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e79950c8948e8196f4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]